In [1]:
from ipywidgets import interact, interact_manual, widgets, Label
import numpy as np
import functools
from time import time, sleep
from os import system, listdir
import h5py
from notebook import notebookapp

In [2]:
s_url='https://github.com/patrickhaddadteaching/TRNG_ex2/raw/main/data/data_for_ex2.zip'
a=system('wget %s > test.log'%s_url)
s_host_url='https://hub.gke2.mybinder.org'

#a=system('cp zips/data_for_ex2.zip . > test.log')
#s_host_url='http://localhost:9999'

a=system('unzip data_for_ex2.zip >> test.log')
a=system('rm -Rf data_for_ex2.zip  >> test.log')

In [3]:
servers = list(notebookapp.list_running_servers())
s_base_url=servers[0]['base_url']

In [4]:
hdf5_file = 'data_for_ex2.h5'
data_file = h5py.File(hdf5_file,'r')
v_ratio_period_sample_for_interact=np.array(data_file['parameters/v_ratio_period_sample'][0,:],dtype=np.uint32)
v_jitter_for_interact=data_file['parameters/v_jitter'][0,:]
v_raw_rng_image=data_file['images/rng_image'][0,:]
m_raw_bits_from_one_RO=data_file['raw_bits/m_raw_bits_from_one_RO'][:]
data_file.close()
system('rm -Rf %s'%hdf5_file)
nb_test_per_corner=m_raw_bits_from_one_RO.shape[3]
nb_bits=m_raw_bits_from_one_RO.shape[4]
nb_osc=m_raw_bits_from_one_RO.shape[0]

In [5]:
width_png=547
height_png=400

In [6]:
image_height=350
image_width=int(width_png*image_height/height_png)
raw_bytes_width=100
tests_results_width=300

In [44]:
def run_fips_tests(v_bits):
    len_v_bits=v_bits.shape[0]
    nb_nibble=int((v_bits.shape[0])/4)
    v_nibble=np.zeros((nb_nibble,),dtype=np.uint8)
    for i in range(4):
        v_nibble+=np.array(v_bits[i:4*nb_nibble:4],dtype=np.uint8)*(2**i)

    T1=v_bits.sum()

    T2=0
    for j in range(16):
        fj=(v_nibble==j).sum()
        T2+=((fj-5000.0/16)**2)/(5000.0/16)

    nb_lambda=32
    m_T3=np.zeros((nb_lambda,2),dtype=np.float64)
    for lambda_i in range(nb_lambda):
        word_size_in_bits=lambda_i+3

        nb_words=int(len_v_bits-word_size_in_bits)
        v_words=np.zeros((nb_words,),dtype=np.uint64)
        for i in range(word_size_in_bits):
            v_words+=np.array(v_bits[i:i+nb_words],dtype=np.uint8)*(2**i)    

        patern_0=2**(word_size_in_bits-1)+1
        patern_1=(2**(word_size_in_bits)-2**(word_size_in_bits-1)-2)
        m_T3[lambda_i,0]=(v_words==patern_0).sum()
        m_T3[lambda_i,1]=(v_words==patern_1).sum()

    """
    lambda_T4=34
    v_sum=np.zeros((len_v_bits-lambda_T4,),dtype=np.uint64)
    for i in range(lambda_T4):
        v_sum+=v_bits[i:len_v_bits-lambda_T4+i]

    T4_PS=((v_sum==0) | (v_sum==lambda_T4)).sum()

    """
    v_cnt=np.zeros((2,),dtype=np.uint64)
    cnt_tmp=0
    for i in range(len_v_bits-1):
        if v_bits[i]==v_bits[i+1]:
            cnt_tmp+=1
            if v_cnt[int(v_bits[i+1])]<cnt_tmp:
                v_cnt[int(v_bits[i+1])]=cnt_tmp
        else:
            cnt_tmp=1
                
    v_T4=v_cnt
    
    v_T5=np.zeros((5000,),dtype=np.uint64)
    for tau in range(5000):
        v_T5[tau]=(v_bits[:5000]^v_bits[tau+1:tau+5000+1]).sum()
    return (T1,T2,[[m_T3[0,0],m_T3[0,1]],[m_T3[1,0],m_T3[1,1]],[m_T3[2,0],m_T3[2,1]],[m_T3[3,0],m_T3[3,1]],[m_T3[4,0],m_T3[4,1]],[m_T3[5:,0].sum(),m_T3[5:,1].sum()]],v_T4,v_T5)


In [45]:
cell_height=30
cell_width=70
cell_width_col0=130
cell_width_col1=70
cell_width_collast=90
cell_width_title=cell_width_col0+cell_width_col1+cell_width_collast+cell_width*2

box_layout = widgets.Layout(height='%dpx'%cell_height,width='%dpx'%cell_width,border='1px solid gray',margin='0px 0px 0px 0px',justify_content='center')
box_layout2 = widgets.Layout(height='%dpx'%cell_height,width='%dpx'%(2*cell_width),border='1px solid gray',margin='0px 0px 0px 0px',justify_content='center')
box_layout_col0 = widgets.Layout(height='%dpx'%cell_height,width='%dpx'%(cell_width_col0),border='1px solid gray',margin='0px 0px 0px 0px',justify_content='center')
box_layout_col1 = widgets.Layout(height='%dpx'%cell_height,width='%dpx'%(cell_width_col1),border='1px solid gray',margin='0px 0px 0px 0px',justify_content='center')
box_layout_collast = widgets.Layout(height='%dpx'%cell_height,width='%dpx'%(cell_width_collast),border='1px solid gray',margin='0px 0px 0px 0px',justify_content='center')
box_layout_title = widgets.Layout(height='%dpx'%cell_height,width='%dpx'%(cell_width_title),border='1px solid gray',margin='0px 0px 0px 0px',justify_content='center')



v_items=[]

v_items_h=[]
v_items_h.append(widgets.HTML(value='<p><b>Procedure A of AIS31</b></p>',layout=box_layout_title))
v_items.append(v_items_h)

v_items_h=[]
v_items_h.append(widgets.HTML(value='<p >Statistical Test</p>',layout=box_layout_col0))
v_items_h.append(widgets.HTML('<p>Test Result</p>',layout=box_layout_col1))
v_items_h.append(widgets.HTML('<p>Test value</p>',layout=box_layout2))
v_items_h.append(widgets.HTML('<p>Valid range</p>',layout=box_layout_collast))
v_items.append(v_items_h)

v_items_h=[]
v_items_h.append(widgets.HTML('<p>Monobit test</p>',layout=box_layout_col0))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout_col1))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout2))
v_items_h.append(widgets.HTML('<p>[9655 ; 10345]</p>',layout=box_layout_collast))
v_items.append(v_items_h)

v_items_h=[]
v_items_h.append(widgets.HTML('<p>Poker test</p>',layout=box_layout_col0))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout_col1))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout2))
v_items_h.append(widgets.HTML('<p>[1.03 ; 57.4]</p>',layout=box_layout_collast))
v_items.append(v_items_h)

m_T3_bounds=[[2267,2733],[1079,1421],[502,748],[223,402],[90,223],[90,223]]
for i in range(6):
    v_items_h=[]
    if i==5:
        v_items_h.append(widgets.HTML('<p>Run test &#x3BB &ge; %d</p>'%(i+1),layout=box_layout_col0))
    else:
        v_items_h.append(widgets.HTML('<p>Run test &#x3BB=%d</p>'%(i+1),layout=box_layout_col0))
    v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout_col1))
    v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout))
    v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout))
    v_items_h.append(widgets.HTML('<p>[%d ; %d]</p>'%(m_T3_bounds[i][0],m_T3_bounds[i][1]),layout=box_layout_collast))
    v_items.append(v_items_h)

v_items_h=[]
v_items_h.append(widgets.HTML('<p>Long Run test</p>',layout=box_layout_col0))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout_col1))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout2))
v_items_h.append(widgets.HTML('<p>[0 ; 33]</p>',layout=box_layout_collast))
v_items.append(v_items_h)


v_items_h=[]
v_items_h.append(widgets.HTML('<p>Autocorrelations test</p>',layout=box_layout_col0))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout_col1))
v_items_h.append(widgets.HTML('<p> </p>',layout=box_layout2))
v_items_h.append(widgets.HTML('<p>[2326 ; 2674] </p>',layout=box_layout_collast))
v_items.append(v_items_h)

nb_lines=len(v_items)
v_hb=[]
for i in range(nb_lines):
    v_tmp=v_items[i]
    nb_row=len(v_tmp)
    v_box=[]
    for j in range(nb_row):
        v_box.append(v_tmp[j])
    v_hb.append(widgets.HBox(v_box))
v_box_table=widgets.VBox(v_hb)


sample_period_value_widgets=widgets.SelectionSlider(description=r'\(D\)',options=list(v_ratio_period_sample_for_interact),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)),readout_format='.1f')
m_value_widgets=widgets.SelectionSlider(description=r'\(m\)',options=list(np.arange(nb_osc,dtype=np.uint8)+1),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))
std_dev_value_widgets=widgets.SelectionSlider(description=r'\(\frac{\sigma_{tot}}{T_{1}}\)',options=list(v_jitter_for_interact),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))

rng_image=widgets.Image(value=v_raw_rng_image.tobytes(), format='png',layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%image_width))
space_for_tests_results=widgets.HTML(value='',placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%image_height,width='%dpx'%(raw_bytes_width/6) ))
raw_bytes=widgets.Textarea(value='',rows=5000,placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%(image_width/4)))

float_bar=widgets.FloatProgress(value=0.0,min=0.0, max=100.0,description='',bar_style='info',style={'bar_color': '#ff0000'},orientation='horizontal',layout=widgets.Layout(width='%dpx'%tests_results_width))
val_button=widgets.Button(description = 'GENERATE RANDOM VALUES',layout=widgets.Layout(width='%dpx'%tests_results_width))
val_button.style.button_color = 'gray'
html_download=widgets.HTML(value='',layout=widgets.Layout(width='%dpx'%tests_results_width))

def generate_rnd(b):
    raw_bytes.value=''
    chosen_sample_period=sample_period_value_widgets.value
    chosen_jitter=std_dev_value_widgets.value
    chosen_m=m_value_widgets.value    
    
    index_sample_period=np.where(chosen_sample_period==v_ratio_period_sample_for_interact)[0][0]
    index_chosen_jitter=np.where(chosen_jitter==v_jitter_for_interact)[0][0]

    v_raw_bits=np.zeros((nb_bits,),dtype=np.bool)
    for i in range(chosen_m):
        tmp_rand=np.random.randint(0,nb_test_per_corner,dtype=np.uint32)
        v_raw_bits^=m_raw_bits_from_one_RO[i-1,index_chosen_jitter,index_sample_period,tmp_rand,:]
        
    nb_nibble=int((nb_bits)/4)
    v_nibble=np.zeros((nb_nibble,),dtype=np.uint8)
    for i in range(4):
        v_nibble+=np.array(v_raw_bits[i:4*nb_nibble:4],dtype=np.uint8)*(2**i)
    html_download.value=''
    s_line=''
    for i in range(nb_nibble):
        s_line='%s%X'%(s_line,v_nibble[i])
    raw_bytes.value=s_line   

    randomvalues_file_name='randomvalues.txt'
    fid_rnd_file=open(randomvalues_file_name,'w')
    fid_rnd_file.write(s_line)
    fid_rnd_file.close()
    path_to_download='%s%sfiles/%s'%(s_host_url,s_base_url,randomvalues_file_name)
    s_line_for_download='<p><a href="%s" target="_blank" rel="noopener noreferrer">  Click here to download the random values</a></p>'%(path_to_download)

    
    (T1_o,T2_o,M_T3_o,v_T4_o,v_T5_o)=run_fips_tests(v_raw_bits)
    v_items[2][2].value='<p>%d</p>'%T1_o
    if abs(T1_o-nb_bits/2)<346:
        v_items[2][1].value='<p style="color:#00FF00";> PASS </p>'
    else:
        v_items[2][1].value='<p style="color:#FF0000";> FAIL </p>'    
    
    v_items[3][2].value='<p>%.2f</p>'%T2_o
    if (T2_o<57.4)&(T2_o>1.03):
        v_items[3][1].value='<p style="color:#00FF00";> PASS </p>'
    else:
        v_items[3][1].value='<p style="color:#FF0000";> FAIL </p>'    

    for i in range(6):
        cnt_error=0
        for b in range(2):
            v_items[4+i][2+b].value='<p>%d</p>'%M_T3_o[i][b]
            if (M_T3_o[i][b]<m_T3_bounds[i][1])&(M_T3_o[i][b]>m_T3_bounds[i][0]):
                cnt_error+=1                
        if (cnt_error>0):
            v_items[4+i][1].value='<p style="color:#00FF00";> PASS </p>'
        else:
            v_items[4+i][1].value='<p style="color:#FF0000";> FAIL </p>'
    
    longest_run=v_T4_o.max()
    v_items[10][2].value='<p>Longest run : %d</p>'%(longest_run)
    if (longest_run<34):
        v_items[10][1].value='<p style="color:#00FF00";> PASS </p>'
    else:
        v_items[10][1].value='<p style="color:#FF0000";> FAIL </p>'   

    max_autocor=v_T5_o.max()
    min_autocor=v_T5_o.min()
    v_items[11][2].value='<p>MIN=%d MAX=%d</p>'%(min_autocor,max_autocor)
    if (max_autocor<2674)&(max_autocor>2326)&(min_autocor<2674)&(min_autocor>2326):
        v_items[11][1].value='<p style="color:#00FF00";> PASS </p>'
    else:
        v_items[11][1].value='<p style="color:#FF0000";> FAIL </p>'  
        
    float_bar.value=0
    time_to_sleep=0.01
    for i in range(100):
        sleep(time_to_sleep)
        float_bar.value=(i+1)
    html_download.value=s_line_for_download
val_button.on_click(generate_rnd, remove=False) 
v_box_top=widgets.VBox([widgets.HBox([rng_image,raw_bytes,space_for_tests_results,v_box_table]),widgets.VBox([widgets.HBox([widgets.VBox([m_value_widgets,std_dev_value_widgets,sample_period_value_widgets]),widgets.VBox([val_button,float_bar,html_download])])])])
display(v_box_top)

Widget Javascript not detected.  It may not be installed or enabled properly.
